http://www2.cao.ie/points/l8.php

***
'PHP' indicates that the webpage has been programmed using PHP, a server side scripting language.

In [1]:
#convenient http requests
import requests as rq

# regular expressions
import re

#dates and times
import datetime as dt

# for dataframes import pandas
# v useful to get data from spreadsheets
import pandas as pd

#for downloading files to python
import urllib.request as urlrq



In [2]:
#now = dt.datetime.now()

In [3]:
#datetime function has a data structure called datetime
#field for time, day ,month etc
#dt.datetime.now()
now = dt.datetime.now()
now

datetime.datetime(2021, 12, 11, 12, 38, 35, 197127)

In [4]:
# format now as a string
#if have datetimestamp as separate and unique at the start of the notebook
# then potentially can overwrite but on positive side all files 
# created in logbook run will have the same datetimestamp so neater and can track
#2020 datafile saved as'now' file is equivalent to corresponding 2021 saved as same'now' etc
nowstr = now.strftime('%Y%m%d_%H%M%S')

In [5]:
print(now)

2021-12-11 12:38:35.197127


# 2021 points

http://www2.cao.ie/points/l8.php

***


In [6]:
# fetch the CAO points URL
# resp= response from the URL
# rq= request library
# get= get the url (NB: gets the HTTP version!)
resp = rq.get ('http://www2.cao.ie/points/l8.php')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

## Save the original dataset
#### use the current time (changed to str above) as a filename

In [ ]:
# good programmers use only alphanumeric characters,dash and dot in file names
#create a filepath for the original data
#use year at start to ensure alphabetic sorted when filed
# below creates the EMPTY html file and saves it in a new 'data' folder in active location  ie. \repo\Fun_of..
path2021html = 'data/cao2021_' + nowstr + '.html'

### the server uses the wrong encoding fix it before saving

In [ ]:
#RESP = RESPONSE
#'resp.text' takes the response and converts it to text- but still has original errors
#thus not best way to save THIS original file
#but resp.text is a typical way to save

In [ ]:
# The server uses the wrong encoding, fix it
original_encoding = resp.encoding

# Change to cp1252
resp.encoding = 'cp1252'

In [ ]:
#save the original html file:
# using the updated coding cp1252 as a TEXT file
#this file is not used in the regular expression script but saved as a record 

with open(path2021html, 'w') as f:
    f.write(resp.text)    

In [ ]:
# python looks for html codes such as \n \t
#\n is a new line (common Unix command)
#resp.text

https://stackabuse.com/pythons-itertools-count-cycle-and-chain/

In [ ]:
# how to loop through (iterate through) the lines in request. 
# outputs the raw html
#for line in resp.iter_lines(): 
    #print(line)

In [ ]:
#compile the regular expression for matching lines
# re= regular expression. Each character is a string but together form an instruction.
# r' = 'raw' used in python. It means don't evaluate backslashes in encountered string literals  i.e. \n \t i.e 
#\*? = if compiler encounters a three number pattern followed by an asterisk- that is acceptable and include in the compliler
# the following asterisk ('wild card') means zero or more characters after this are allowed

#re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(.*)')
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')

#shortened RE to simplify i.e. does the line have two alphabetic characters followed by three numbers followed by anything else
#re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

#### cao2021 course count anomalies:

when use re_course = re.compile(r'([A-Z]{2}[0-9]{3})(.*)') course count = 949
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *')  course count = 922

The difference is the simpler 1st regular expression returns courses (code and course title) without an associated points entry e.g. 
AL811,Civil Engineering,,

<img src="images/CAOhtmlscreenshot.png" width="500"/>

A direct comparison of the two files in Notepad ++ identified the following lines returned with the simpler regular expression which accounts for the difference
1.	AL811,Civil Engineering,,
2.	CK114,Social Science (Youth and Community Work),#,
3.	CK115,Social Work - Mature Applicants only,,
4.	CK791,Medicine  - Graduate Entry (GAMSAT required),#60*,#60*
5.	DB503,Computing (Software Development),,
6.	DB504,Computing (Web and Mobile Technology),,
7.	GC402,Business (HRM),,
8.	ID001,Accounting and Finance,,
9.	ID002,Business Studies,,
10.	RC101,Medicine - Graduate Entry (GAMSAT required),,
11.	DN301,Veterinary Medicine - Graduate Entry (GAMSAT requi,,
12.	DN401,Medicine - Graduate Entry (GAMSAT required),#61*,
13.	DN411,Radiography - Graduate Entry,#,
14.	DK885,Agri-Food Production,,
15.	GA892,Community Development and Youth Work (Level 8),,
16.	GA983,Furniture and Digital Manufacturing,,
17.	LM101,Medicine  - Graduate Entry (GAMSAT required),#57*,
18.	MU002,Theology,,
19.	MH002,Education - Primary Teaching - Gaeltacht Applicant,,
20.	MH103,Music,#,
21.	MH116,Community and Youth Work (Full-Time),#,
22.	MH801,Early Childhood - Teaching and Learning (part-time,AQA,AQA
23.	MH802,Community and Youth Work (part-time  in service),,#
24.	MH803,Local Studies/Community Studies (part-time  evenin,,#
25.	SG349,Electronics and Self Driving Technologies,,
26.	TL842,Construction Management,,
27.	TL889,Counselling with Addiction - Mature applicants onl,,

Therefore the longer regular expression which matches to lines with course points is the recommended course to use. <br>
However other year CAO points to be checked for consistency with courses.

# Notes: 
### Open two csv files in notepad and compare (plugins).
- This could be used to compare between years and identify added, missing courses, anomalies that need the RE modified

<br>

**error on the server= way to fix**

match only the lines we want -the ones representing courses 
 technically the server says we should decode as:
 
content-type: text/html; charset: iso-8859-1

  but
    -one line uses /x96 which isn't defined in iso-8859-1
    -therefore we used the similiar decoding standard (codec) cp-1252 which is very similar but includes \x96

### loop through the lines of the response
***

In [ ]:
#the filepath for the csv file- which is the output of the below script
#path2021csv = 'data/cao2021_csv' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'
path2021csv = 'data/cao2021_csv' + nowstr + '.csv'

In [ ]:
#PASTED IN FROM IMcL NOTEBOOK FROM COURSEBOOK
# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021csv, 'w') as f:
    # Write a header row.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    # Loop through lines of the response.
    for line in resp.iter_lines():
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            print(line)
            # The course code.
            course_code = dline[:5]
            # The course title.
            course_title = dline[7:57].strip()
            # Round one points.
            course_points = re.split(' +', dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            # Join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')

# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")

In [ ]:
df2021 = pd.read_csv(path2021csv, encoding='cp1252')

In [ ]:
df2021[20:50]

In [ ]:
#the filepath for the csv file- which is the output of the below script
path2021csvB = 'data/cao2021_csvB' + nowstr + '.csv'

In [ ]:
#for the below if used in assessment should be cleaned up. Check regular expressions are working properly
# if want to save output of this then need a filepath and file format like the original data above

no_lines = 0
#open the csv file for writing
with open(path2021csvB, 'w') as f:
    # write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
   # f.write(resp.text)   
    #loop through the lines of the response content
    for line in resp.iter_lines():
        #decode the line using the encoding not flagged by the response from CAO
        dline = line.decode('cp1252')
        #match only the lines representing the requirements of the regular expression (i.e. line containing course)
        if re_course.fullmatch(dline):
            #add one to the line counter
            no_lines = no_lines + 1
            print(line)
            # to check the output- can be commented out as required

        # version 1 using regular expression
        # csv_version = re_course.sub(r'\1,\2,\3,\4', dline)
        #version 1 using re commented out
        #print(csv_version) no need to print so commented out
        #alternate way than regular expression is to use python in built split function
        # 'split' splits the line
        #(+ ) instructs to do this where two or more spaces- NB the spaces represented before the '+' below are v important
            linesplit = re.split('  +', dline)
            f.write(','.join(linesplit) + '\n') 
print(f"total number of lines is {no_lines}.")  
# long outputs are a pain in github as will render without the abiliity to scroll. 
# Therefore don't print it all out.   
# Can use scikit learn. Suggestion below


In [ ]:
#df2021B = pd.read_csv(path2021csvB, encoding='cp1252')

In [ ]:
#df2021B

#### regular expression syntax
https://www.w3schools.com/python/python_regex.asp

https://docs.python.org/3/library/re.html

#### how to use split function
https://pynative.com/python-regex-split/#h-how-to-use-re-split-function

syntax: re.split(pattern, string, maxsplit=0, flags=0)

In [ ]:
# linspace:

#The numpy.linspace() function returns number spaces evenly w.r.t interval. Similar to numpy.arrange() function but instead of step it uses sample number. 
# Syntax : numpy.linspace(start,stop,num = 50,endpoint = True,retstep = False,dtype = None)
#e.g.:
# import numpy as np
# print("B\n", np.linspace(2.0, 3.0, num=5, retstep=True), "\n")
# To evaluate sin() in long range
# x = np.linspace(0, 2, 10)
# print("A\n", np.sin(x))
# https://www.geeksforgeeks.org/numpy-linspace-python/


# print out every 'th value in an array:
# e.g. for a super long list i.e. here 10000 print only evertyime i is a multiple of 1000
# i = 0 (not needed)
# for i in x  (not needed)

import numpy as np
x = np.linspace(-20.0, 10.0,10000)
for i in range (len(x)):
    if (i % 1000) == 0:
            print (f"{x[i]:.1}")
# this will avoid the length cell especially in github notebook renderer and nbv viewer

<br>

#### helper function

In [ ]:
# takes in s (str) - checks is 1st character #? 
#if yes then stores in portfolio a
# otherwise it stores an empty string
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
        random = ''
        #checks the last character  if its an asterisk then it stores 
        if s[-1] == '*':
            random = '*'
            # loops through each characters in string for i in s
            # + is string concatenation 
            # + is not addition
        points = ''
        for i in s:
            if i.isdigit():
                points = points + 1
                return [points, portfolio, random]
            
# 'would recommend changing course _points[0], course_points[1]] into arrays with three things in them
#hash character , empty string, points ,random has-
# maybe don't need if read in pandas'

#### Extra lines in html- how to reconcile the # lines of html vs the csv
- i.e. total number of lines output in jupyter code
- open VSC
- delete all the lines that don't want
- delete preamble & sections within colleges

####  find \n\n and replace with \n
in RE turned on : ^ looks for start of line $ looks for end of line <br>
cant find and replace with these but can find S&F of lines


#### alternate way to sort
- different to RE but similar result
- more manual


In [ ]:
#the filepath for the alternate csv file- which is the output of the below script that uses a more 1st principles approach
#path2021csv2 = 'data/cao2021_csv(2)' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'
path2021csv2 = 'data/cao2021_csv(2)' + nowstr + '.csv'

In [ ]:
# older version of code

no_lines = 0
#open the csv file for writing
with open(path2021csv2, 'w') as f:
    # write a header row
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    #debug:
            ##print(len(linesplit), linesplit ,dline)
            #this will show you how the RE is splitting 
            #i.e. expect '4' for len(length) of linesplit, then in square brackets how it splits 'xxx', then outside square brackets the string- 'dline'
            
            #print out just the course code # :5b means simply print out the characters up to position 5 (0-4 in python)
            #course_code = dline[:5]
            #print (course_code)
            
            #print out just the course title- starts at 7n in..trial and error re run until get longest course title
            #course_title = dline[7:57]
            #print (course_title)     
            
            #round one points - prints out both round 1 and round 2  Note: can't separate 'round' and '1'
            #course_round1 = dline[60:]
            #print (course_round1)
            
            #can edit the RE and re-run these to debug further
            
            #further debugging
            #split on the space. Has a look for one ore more space and splits it
            #course_points = re.split('  +' , dline[60:])
            #print(f"'{course_code} {course_points}'")
            
            #last line is returning anything but two substrings back- create an exception
           # if len(course_points) != 2:
                #course_points = course_points[:2]
               # join the fields usign a comma
               # linesplit = [course_code,course_title, course _points[0], course_points[1]]
                
            #print(','.join(linesplit))
            #f.write(csv_version + '\n') - saving of old RE way
            #rejoin the substrings with commas in-between
            #f.write(','.join(linesplit) + '\n')
            
            #result - output is a csv file!
    for line in resp.iter_lines():
        dline = line.decode('cp1252')
        if re_course.fullmatch(dline):
            no_lines = no_lines + 1
            course_code = dline[:5]
            course_title = dline[7:57].strip()
            course_points = re.split('  +' , dline[60:])
            if len(course_points) != 2:
                course_points = course_points[:2]
            linesplit = [course_code,course_title, course_points[0], course_points[1]]
            f.write(','.join(linesplit) + '\n') 
            print(line) # to check the output- can be commented out as required
print(f"total number of lines is {no_lines}.")

# output path2021 is a csv file
#note pandas coud convert html to csv but better to use own code and save as csv- use pandas thereafter

stripstrings = ' abc     ' <br>
**s.strip()** strips the white space** <br>

<br>

**NB** it was verfied as pof  xx yy zz that there were  949 courses exactly in the CAO 2021 points list

In [ ]:
#old code
#how to loop throught the linesv in request. iterate through the lines in request using a regular expression
#for line in resp.iter_lines():
   # if re.match('[A-Z]{2}[0-9]{3}  .* *(0-9){3} *', line.decode('utf-8') ):
               # print(line)

In [ ]:
df2021B = pd.read_csv(path2021csv2, encoding='cp1252')

In [ ]:
df2021B

<br>

# 2020 points

http://www.cao.ie/index.php?page=points&p=2020

***

### download and parse the excel spreadsheet using pandas

In [ ]:
#create a variable for the url to keep it neat
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

In [ ]:
#download and parse the excel spreadsheet
#skip the 1st 10 preamble rows
df2020 = pd.read_excel (url2020, skiprows = 10)
# however it reads the 1st 10 preamble headings 
# could save file and manually delete 1st 10 rows but better to use pandas functionality

In [ ]:
df2020

In [ ]:
#spot check of data (row)
# use loc or iloc functionality to check data
df2020.iloc[12:15]
#check the final line in the spreadsheet
# reminder 1st 11 rows are heading #excel identifies 1st row as zero
#df2020.iloc[753]
# can also count backwards to last row
#df2020.iloc[-1]

<br>

### save original file 

***

In [ ]:
# Create a file path for the pandas data.
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

# retrieve the original excel data from CAO website and save to new xlsx file (time stamped)
urlrq.urlretrieve (url2020, pathxlsx)

#pathxlsx = 'data/cao2020_' + now.strftime('%Y%m%d_%H%M%S')  + '.xlsx'
#now.strftime('%Y%m%d_%H%M%S')
#save the original html file
#with open(pathxlsx, 'w') as f:
   # f.write(resp.text)   

<br>

#### create a filepath for pandas data
***


In [ ]:
#save the output to a csv file
#path2020 = 'data/cao2020_' + now.strftime('%Y%m%d_%H%M%S')  + '.csv'
path2020 = 'data/cao2020_' + nowstr + '.csv'


In [ ]:
# Save pandas data frame to disk.
df2020.to_csv(path2020)

<br>

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.htmlpandas.read_excel <br> function signature in pandas excel webpage: (io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=None, mangle_dupe_cols=True, storage_options=None)¶
skiprows is what we need

## fetch earlier year points data and save in data files

<br>

## 2019 points
-as pdf option here is to read do it manually before getting pandas to read it as a csv

http://www.cao.ie/index.php?page=points&p=2019

***

-Note the 2019 and earlier points are in pdf format<br>
Main points index url: <br>
https://www.cao.ie/index.php?page=points
direct link tp pointsd tablr (pdf) <br>
http://www2.cao.ie/points/lvl8_19.pdf


<br>

### steps to reproduce the 2019 editable:
****


Manual way of extracting:
1. save pdf in repository
2. open original with MS Word (may need to 'choose other app to open' in dropdown
3. MS Word converts pdf to Word
4. Save file as xxxxx.docx format
5. Resave a copy to edit in repo as xxxxx_edited.docx
6. Delete headers and footers
7. delete preamble on page 1
8. Ctrl 'A', Ctrl 'C' to copy all data in the 21 separate tables
9. Open notepad++ and paste 'Ctrl V'


### Steps to reproduce \textcolor{red}{red} <br>
<span style="color:blue">NB: for the video I truncated the CSV file to save time. <br></span>

<mark>After the video, I went back and changed some of the following to make it cleaner.<mark>
- Download original pdf file.
- Open original pdf file in Microsoft Word.
- Save Microsoft Word's converted PDF in docx format.
- Re-save Word document for editing.
- Delete headers and footers.
- Delete preamble on page 1.
- Select all and copy.
- Paste into Visual Studio Code.
- Remove HEI name lines and black lines.
- Change column heading "COURSE AND INSTITUTION" to "Course".
- Change backticks to apostrophes.
- Replaced double tab charater at on line 28 with single tab.
-Delete tabs at end of lines 604, 670, 700, 701, 793, and 830.


### Creating CSV file from the original pdf

- The original pdf was downloaded and saved as a pdf in repository
- The original file was opened with MS Word ass follows:
- Select file > open with > other programs > MSWord >
- open for editing
- Save the editable file in repository under data as as xxxxx.docx format: 
- Resaved a copy to edit in repo as cao2019_edited.docx
- Deleted headers and footers
- deleted the preamble on page 1
- Ctrl 'A', Ctrl 'C' to copy all data in the 21 separate tables
- Opened notepad++ and pasted in the content of the 21 tables 'Ctrl V'

#### manual editing in notepad ++
 
- Removed HEI name lines and black lines
- Removed all Irish fadas: e.g. Gn**ó** agus Gaeilge 
- Delete blank lines
- Change backticks to apostrophes
- Remove all Irish fadas: e.g. Gn**ó** agus Gaeilge 
- find \\t replace with \t finds the double tabs and removes them
- Replaced double tab charater at on line 28 with single tab.
- Delete tabs at end of lines 604, 670, 700, 701, 793, and 830.
- save file


Note: Institutions not required for data frame as institution is identifiable from the course code 1st two letters.<br>  Thus the below not required: <br>
1. Select the 'College/Institute' i.e. 'Athlone Institute of Technology' and cut
2. Move cursor to in front of AIT 1st course line
3. Hold down ALT key and drag cursor down to all AIT courses
4. Ctrl V to paste - click 'tab' key to create tabs
5. Put double quotes around college names as they have commas [select start of string with cursor-alt-type "]
6. Also: \t  in extended file 'replace all' with comma: replaces all the tabs with commas <br> however not recommended here as commas elsewhere will confuse. <br> best to leave tabs as delimiters  e.g. in effect its as 'tsv' tab serapated file but can save as csv


In [7]:
df2019 = pd.read_csv('data/cao2019_20211102_191031_edited.csv', sep ='\t') 

In [8]:
df2019

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
0,,NaN,NaN,NaN,NaN
1,Athlone Institute of Technology,AL801,Software Design with Virtual Reality and Gaming,304,328.0
2,Athlone Institute of Technology,AL802,Software Design with Cloud Computing,301,306.0
3,Athlone Institute of Technology,AL803,Software Design with Mobile Apps and Connected...,309,337.0
4,Athlone Institute of Technology,AL805,Network Management and Cloud Infrastructure,329,442.0
...,...,...,...,...,...
974,Waterford Institute of Technology,WD200,Arts (options),221,296.0
975,Waterford Institute of Technology,WD210,Software Systems Development,271,329.0
976,Waterford Institute of Technology,WD211,Creative Computing,275,322.0
977,Waterford Institute of Technology,WD212,Recreation and Sport Management,274,311.0


In [9]:
#spot check of data (row)
# use loc or iloc functionality to check data
df2019.iloc[12:15]
#check the final line in the spreadsheet
# reminder 1st 11 rows are heading #excel identifies 1st row as zero
#df2019.iloc[753]
# can also count backwards to last row
#df2019.iloc[-1]

,HEI,Course Code,INSTITUTION and COURSE,EOS,Mid
12,Athlone Institute of Technology,AL839,Microbiology,322,371.0
13,Athlone Institute of Technology,AL840,Pharmaceutical Sciences,328,411.0
14,Athlone Institute of Technology,AL841,Athletic and Rehabilitation Therapy,423,430.0


In [10]:
# fetch the CAO points URL.  The url for earlier years takes you to a pdf version
# However the python code to fetch appears to work
resp = rq.get ('http://www2.cao.ie/points/lvl8_19.pdf')
#have a quick peek
resp
# resp [200] means html response is in order.  [404] if returns means it is not found .  These are http codes

ChunkedEncodingError: ("Connection broken: ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)", ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [ ]:
#path2019 = 'data/cao2019_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'
path2019 = 'data/cao2019_' + nowstr + '.html'
#now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path2019, 'w') as f:
    f.write(resp.text)   

In [12]:
df2019 = pd.read_csv('data/cao_2019.csv', sep ='\t') 

In [13]:
df2019

,Course Code,COURSE,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


#### fetch 2018

**propose to go through the same process as 2019**

In [ ]:
resp = rq.get ('http://www2.cao.ie/points/lvl8_18.pdf')
resp

In [ ]:
path2018 = 'data/cao2018_' + now.strftime('%Y%m%d_%H%M%S')  + '.html'
now.strftime('%Y%m%d_%H%M%S')
#save the original html file
with open(path2018, 'w') as f:
    f.write(resp.text) 

###How to convert pdf to csv in python- further reading: <br>
-https://stackoverflow.com/questions/49560486/how-to-convert-pdf-to-csv-with-tabula-py

<br>

### Concatenate and Merge dataframes

how to concatenate with pandas <br>
note 'ignore_index'= True <br>
https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html <br>
how to manage duplicates in a dataframe <br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html <br>
how to return uniques in dastaframe columns <br>
https://pandas.pydata.org/docs/reference/api/pandas.Series.unique.html <br>
how to drop duplicates in dataframe <br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

In [ ]:
#extract the required columns (course and points) from the dataframe
courses2021= df2021[['code' , 'title']]

In [ ]:
courses2021

In [ ]:
courses2020= df2020[['COURSE CODE2' , 'COURSE TITLE']]
# change the title of the columns to match 2021
courses2020.columns = ['code' ,'title']

In [ ]:
courses2020

In [ ]:
#merge or concatenate the two datasets into one dataframe
#set ignore_index = True (default is False)
allcourses = pd.concat([courses2021, courses2020], ignore_index=True)

In [ ]:
allcourses

In [ ]:
# returns a false for every row that are not duplicated
allcourses.duplicated()

In [ ]:
#finds all extra copies of duplicated rows
#duplicated by default doesnt set the 1st iteration of the matching rows as true (returns false)
allcourses[allcourses.duplicated()] 

In [ ]:
allcourses[allcourses.duplicated(keep = False)]

In [ ]:
#returns back the dataframe with all duplicates dropped
allcourses.drop_duplicates()

In [ ]:
# just look for duplicates of the code- as the course titles are likely different from year to year
allcourses[allcourses.duplicated(subset =['code'])]

In [ ]:
#drop duplicates based on the same code
allcourses.drop_duplicates(subset=['code'])

In [ ]:
#drop duplicates based on the same code
#creates a new dataframe as by default-this is identified by 'inplace=False'
#ignore_index cleans up the duplicates in the index i.e. counts just one of duplicates indexes
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [ ]:
#as we used inplace=True then we have permanentlty unduplicated the dataframe for codes
#when run allcourses this time the default dtaframe has no course code duplicates
allcourses

reset index in pandas dataframe<br>
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

In [ ]:
#sort by courses
allcourses.sort_values ('code')

In [ ]:
allcourses.loc[0]

In [ ]:
allcourses.iloc[0]

<br>

## Join to the Points 
***
<br>


worth knowing: pandas dataframe.join
how:
**main event is left**
- left join: - whatever df is on the left (join call) the other is on the right . keeps everything from the left
- only join: keeps values in BOTH dataframes
- right join: keeps everything in the right dataframe
- full-outer join merges everything- (have a lot of NaN's)

In [ ]:
#set the index to the code column
df2021.set_index ('code', inplace=True)
df2021.columns = ['title' , 'points_r1_2021', 'points_r2_2021']
df2021

In [ ]:
#pull in the points from 2021 and 2020
#see pandas concatenate reference above

In [ ]:
#set the column as the index
allcourses.set_index('code', inplace =True)

In [ ]:
#just merge what you need from the 2021 dataframe i.e. code and points
#where NaN it defines as code that was not run in 2021
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

In [ ]:
# just need to join the pointd from 2020
df2020_r1= df2020 [['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_R1_2020']
df2020_r1

In [ ]:
# set the index to the code column
df2020_r1.set_index ('code', inplace=True)
df2020_r1                 

In [ ]:
#just merge what you need from the 2021 dataframe i.e. code and points
#where NaN it defines as code that was not run in 2021
#join2020 points to all courses
#allcourses = allcourses.join(df2021[['pointsR1']])
#allcourses

In [ ]:
# join 2020 points to all courses
allcourses = allcourses.join(df2020_r1)
allcourses

https://www.prepressure.com/library/technology/ascii-binary-hex

http://www.i18nqa.com/debug/table-iso8859-1-vs-windows-1252.html

https://docs.python.org/3/library/datetime.html

https://www.ibm.com/ie-en/products/spss-statistics-gradpack

In [ ]:
#strftime turns datetime to strings
#strptime turns strings into datetime

#### SPSS
SPSS currently owned by IBM <br> 
All click and point <br>
Not as powerful as Python <br>
'Level of Measurement'<br>
Typically dealing with one table of data <br>
Similar functionality but less friendly than Excel <br>
clone function (open source) called PSPP <br>
Laerd: Really high quality stastistical explanations <br>
 to e.g. get a good explanation of e..g. One Way Anova type 'Laerd One Way Anova' etc

https://www.ibm.com/ie-en/products/spss-statistics-gradpack <br>
https://www.gnu.org/software/pspp/ (PSPP) <br>
https://statistics.laerd.com/ (Laerd) <br>

### Categorical (Variables)
- similar to qualitative data
- sex
- job description
- opinion
##### Nominal: no inherent order
- job category: doctor, accountant
##### Ordinal: there is a meaningful order but not a measurable distance between categories
-items can be ranked <br>
- 1st, 2nd, 3rd<br>
- strongle disagree, disagree, neutral, strongly agree<br>
- does not allow for relative degree of difference between them
### Scale (Variables)
typically as INTERVAL or RATIO <br>
values can be ordered <br>
suggest a distance between values <br>
age, salary <br>
##### INTERVAL
-20C is not twice as hot as 10C<br>
##### RATIO
-20K is not twice as hot as 10K<br>
-true zero point
-numercial values can be transfprmed by multiplying each value by a constant


##### Further reading
1 Stanley Stevens: Invented classifications related to:<br>
Level of Measurement <br>
Scales of Measurement
 his original 1946 paper can still be sourced
2. Aelph Number
3. Cantors Diagonal Argument <br>
certain infite sets 0,1,2,3 v 2,4,6,8.. must have same size as can be paired <br>
 but are actually not the same size
4. Ordinal Numbers <br>
natural numbers 

- NOMINAL/ORDINAL/INTERVAL/RATIO <br>

-https://symbiosiscollege.edu.in/assets/pdf/e-learning/tyba/Economics/Scales-of-Measurement-6.pdf
